In [1]:
import cv2
import pyxdf
import numpy as np

In [2]:
xdf_file = 'tobii_test_trial_lsl.xdf'

streams, header = pyxdf.load_xdf(xdf_file)

In [3]:
gaze_stream = None
for stream in streams:
    if stream['info']['name'][0] == 'Glasses3_Gaze':
        gaze_stream = stream
        break

In [4]:
if gaze_stream is None:
    print("Gaze data not found.")

In [ ]:
print(gaze_stream.keys())

dict_keys(['info', 'footer', 'time_series', 'time_stamps', 'clock_times', 'clock_values'])


In [13]:
for channels in gaze_stream['info']['desc'][0]['channels'][0]['channel']:
    print(channels['label'][0])

local_ts
gaze_ts
x-coordinate
y-coordinate


In [46]:
data = gaze_stream['time_series']
timestamps = gaze_stream['time_stamps']
clocktimes = gaze_stream['clock_times']
clockvalues = gaze_stream['clock_values']

print(data.shape)
print(timestamps.shape)
print(type(clocktimes))
print(len(clocktimes))
print(type(clockvalues))
print(len(clockvalues))

(4081, 6)
(4081,)
<class 'list'>
36
<class 'list'>
36


In [42]:
print(data[0])
print(data[4080][0] - data[0][0])
print(timestamps[4080]-timestamps[0])
print(data[0][0])
print(timestamps[0])

print()
print(clocktimes[0])
print(clockvalues[0])

[2.3218227e+05 1.7595304e+09 1.4440000e+03 3.7600000e+02 7.5250000e-01
 3.4889999e-01]
176.98438
176.97457303776173
232182.27
232182.26906510722

232187.9084268
-1.5199999324977398e-05


In [25]:
px = 7.5250000e-01 * (1920-1)
py = 3.4889999e-01 * (1080-1)
print(px, py)

1444.0475 376.46308921


In [23]:
cap = cv2.VideoCapture('scenevideo.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(frame_width, frame_height)

1920 1080


In [5]:
import cv2
import pyxdf
import numpy as np

# Load XDF file 
xdf_file = 'tobii_test_trial_lsl.xdf'
streams, header = pyxdf.load_xdf(xdf_file)

# Load Tobii gaze coordinates
gaze_stream = None
for stream in streams:
    if stream['info']['name'][0] == 'Glasses3_Gaze':
        gaze_stream = stream
        break

if gaze_stream is None:
    raise ValueError("Glasses3_Gaze stream not found in XDF file.")

data = gaze_stream['time_series']
timestamps = gaze_stream['time_stamps']

# Gaze data: [local_ts, gaze_ts, pixel_x, pixel_y, norm_x, norm_y]
gaze_x = data[:, 4]
gaze_y = data[:, 5]

# Load video
video_path = 'scenevideo.mp4'
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_time = 1.0 / fps  # seconds per frame
cv2.namedWindow('Gaze Overlay', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Gaze Overlay', 960, 540)

# Play video with gaze overlay
timestamps = timestamps - timestamps[0]
gazestartfirst = False
if gazestartfirst:
    offset = 8.0
    valid_indices = timestamps >= offset
    timestamps = timestamps[valid_indices] - offset
    gaze_x = gaze_x[valid_indices]
    gaze_y = gaze_y[valid_indices]

frame_idx = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break  # end of video
    
    current_video_time = frame_idx * frame_time
    print(current_video_time)

    # Find the nearest gaze
    gaze_index = np.argmin(np.abs(timestamps - current_video_time))
    gx, gy = gaze_x[gaze_index], gaze_y[gaze_index]
    
    # Convert gaze coordinates to pixel coordinates
    h, w, _ = frame.shape
    px = int(gx * w)
    py = int(gy * h)

    # Draw gaze overlay
    cv2.circle(frame, (px, py), 20, (0, 0, 255), 2)

    # Show frame
    cv2.imshow('Gaze Overlay', frame)
    if cv2.waitKey(int(frame_time * 1000)) & 0xFF == ord('q'):
        break

    frame_idx += 1

cap.release()
cv2.waitKey(500)
cv2.destroyAllWindows()


0.0
0.04007253688719657
0.08014507377439314
0.12021761066158972
0.1602901475487863
0.20036268443598287
0.24043522132317943
0.280507758210376
0.3205802950975726
0.36065283198476916
0.40072536887196575
0.4407979057591623
0.48087044264635886
0.5209429795335554
0.561015516420752
0.6010880533079486
0.6411605901951452
0.6812331270823417
0.7213056639695383
0.7613782008567349
0.8014507377439315
0.841523274631128
0.8815958115183246
0.9216683484055211
0.9617408852927177
1.0018134221799142
1.0418859590671108
1.0819584959543074
1.122031032841504
1.1621035697287005
1.2021761066158971
1.2422486435030937
1.2823211803902903
1.322393717277487
1.3624662541646835
1.40253879105188
1.4426113279390766
1.4826838648262732
1.5227564017134698
1.5628289386006664
1.602901475487863
1.6429740123750594
1.683046549262256
1.7231190861494525
1.7631916230366491
1.8032641599238457
1.8433366968110423
1.8834092336982389
1.9234817705854355
1.963554307472632
2.0036268443598284
2.043699381247025
2.0837719181342216
2.123844455

In [ ]:
import json

file_path = "gazedata"

timestamps = []
gaze2d_x = []
gaze2d_y = []

with open(file_path, "r") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue  # skip blank lines
        try:
            obj = json.loads(line)
            if obj.get("type") == "gaze":
                gaze2d = obj["data"].get("gaze2d", [])
                if gaze2d and len(gaze2d) == 2:
                    timestamps.append(obj["timestamp"])
                    gaze2d_x.append(gaze2d[0])
                    gaze2d_y.append(gaze2d[1])
                else:
                    continue
        except json.JSONDecodeError as e:
            print("Skipping line due to JSON error:", e)

print(len(timestamps), len(gaze2d_x), len(gaze2d_y))


13020 13020 13020


In [7]:
print(timestamps[0],timestamps[-1])

0.00396 169.370339


In [9]:
import cv2
import json
import numpy as np

# Load gaze data directly from Tobii
file_path = "gazedata"

timestamps = []
gaze2d_x = []
gaze2d_y = []

with open(file_path, "r") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue  # skip blank lines
        try:
            obj = json.loads(line)
            if obj.get("type") == "gaze":
                gaze2d = obj["data"].get("gaze2d", [])
                if gaze2d and len(gaze2d) == 2:
                    timestamps.append(obj["timestamp"])
                    gaze2d_x.append(gaze2d[0])
                    gaze2d_y.append(gaze2d[1])
                else:
                    continue
        except json.JSONDecodeError as e:
            print("Skipping line due to JSON error:", e)

timestamps = np.array(timestamps, dtype=float)
gaze2d_x = np.array(gaze2d_x, dtype=float)
gaze2d_y = np.array(gaze2d_y, dtype=float)

# Load video
video_path = 'scenevideo.mp4'
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_time = 1.0 / fps  # seconds per frame
cv2.namedWindow('Gaze Overlay', cv2.WINDOW_NORMAL)
cv2.resizeWindow('Gaze Overlay', 960, 540)

# Play video with gaze overlay
frame_idx = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break  # end of video
    
    current_video_time = frame_idx * frame_time

    # Find the nearest gaze
    gaze_index = np.argmin(np.abs(timestamps - current_video_time))
    gx, gy = gaze2d_x[gaze_index], gaze2d_y[gaze_index]
    
    # Convert gaze coordinates to pixel coordinates
    h, w, _ = frame.shape
    px = int(gx * w)
    py = int(gy * h)

    # Draw gaze overlay
    cv2.circle(frame, (px, py), 20, (0, 0, 255), 2)

    # Show frame
    cv2.imshow('Gaze Overlay', frame)
    if cv2.waitKey(int(frame_time * 1000)) & 0xFF == ord('q'):
        break

    frame_idx += 1

cap.release()
cv2.waitKey(500)
cv2.destroyAllWindows()